In [2]:
pip install matplotlib

Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import rdMolDescriptors

from keras.layers import Input, Dense
from keras.models import Model
from keras.utils import plot_model


from tensorflow.keras.models import Sequential, save_model, load_model
from tensorflow.keras.layers import Dense, Flatten, LSTM, Embedding, Bidirectional, TimeDistributed, Reshape
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, Dropout
import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

2023-03-06 12:40:17.875609: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
# importing data
df = pd.read_csv('data.csv')
  
# head of the data
print(df.head())

   counts                  smiles property  value
0       0             [*]CC([*])C      Eat  -5.14
1       1             [*]CC([*])F      Eat  -5.18
2       2          [*]CC([*])(F)F      Eat  -5.21
3       3       [*]C(F)C([*])(F)F      Eat  -5.11
4       4  [*]CCC(F)(F)C([*])(F)F      Eat  -5.21


In [6]:
smiles = df['smiles']
print(smiles)
type(smiles)

0                                      [*]CC([*])C
1                                      [*]CC([*])F
2                                   [*]CC([*])(F)F
3                                [*]C(F)C([*])(F)F
4                           [*]CCC(F)(F)C([*])(F)F
                           ...                    
6260    [*]C(F)(F)C(F)(F)C(S1)=CC=C1C(S2)=CC=C2[*]
6261                    [*]C(F)(F)C(F)(F)C(=O)O[*]
6262          [*]C(F)(F)C(=O)C(F)(F)C(S1)=CC=C1[*]
6263        [*]C(F)(F)C(=O)C(F)(F)C(C=C1)=CC=C1[*]
6264                        [*]C(F)(F)OC(F)(F)O[*]
Name: smiles, Length: 6265, dtype: object


pandas.core.series.Series

In [7]:
# Find the number of missing values
missing_values = df.isnull().sum().sum()
print("Number of missing values:", missing_values)


Number of missing values: 47


In [8]:
# Delete the missing values
df = df.dropna()

In [9]:
# Verify that there are no more missing values
assert df.isnull().sum().sum() == 0

missing_values = df.isnull().sum().sum()

print("No more missing values in the dataset.")
print("Number of missing values:", missing_values)

No more missing values in the dataset.
Number of missing values: 0


In [28]:
X = df['value']
print(X)

0      -5.14
1      -5.18
2      -5.21
3      -5.11
4      -5.21
        ... 
6260    4.36
6261    3.31
6262    4.05
6263    3.57
6264    3.80
Name: value, Length: 6218, dtype: float64


In [11]:
train_df,test_df = train_test_split(df,random_state = 104,test_size = 0.25,shuffle = True)
print('train_df : ')
print(train_df.head())
print('test_df : ')
print(test_df.head())

train_df : 
      counts                                  smiles property    value
4775    4775   [*]c1ccc(-c2nc3cc4nc([*])oc4cc3o2)cc1      Eea   3.0398
962      962               [*]CC([*])C(=O)OCCCSCCC#N      Egc   5.4080
783      783            [*]C(=O)c1ccc(N([*])CCCC)cc1       Xc  71.5600
5768    5768  [*]NC(=O)C(C=C1)=CC=C1C(C=C2)=CC=C2[*]       nc   2.0808
937      937                   [*]CC([*])(C)C(=O)OCC      Egc   6.4148
test_df : 
      counts                                   smiles property   value
854      854                   [*]CC([*])c1ccc(C)cc1C      Egc  4.8585
3165    3165                         [*]CCCNC(=O)O[*]      Egc  6.2603
2567    2567  [*]Oc1ccc(Oc2ccc(Oc3cccc([*])n3)cc2)cc1      Egc  4.1433
1592    1592    [*]CCC(C)(C)CC(C)CNC(=O)CCCCC(=O)N[*]      Egc  5.8095
5601    5601       [*]CC(C=C1)=CC=C1C(C=C2)=CC=C2O[*]       nc  1.8665


In [12]:
# MOL = pd.DataFrame(['C(OC(=O)C(C[*])[*])C(F)(F)F',
# 'O(C(=O)C(C[*])[*])CCOC(C(F)(F)F)C(F)(F)F',
# 'O(C(=O)C(C[*])[*])CCOC(C(F)(F)F)(C(F)(F)F)C(F)(F)F',
# 'O(C(=O)C(C[*])[*])CCOC',
# 'O(C(=O)C(C[*])[*])CCO[H]',
# 'O(C(=O)C(C[*])[*])CC[S](C)=O'])[0].apply(Chem.MolFromSmiles)

#MOL = smiles.apply(Chem.MolFromSmiles)
#MOL = pd.DataFrame(['[*]CC([*])C','[*]CC([*])F','[*]CC([*])(F)F','[*]CC([*])(F)F'])[0].apply(Chem.MolFromSmiles)
MOL = pd.DataFrame(['[*]CC([*])C',
                    '[*]CC([*])F',
                    '[*]CC([*])(F)F',
                    '[*]CC([*])(F)F',
                    '[*]C(F)C([*])(F)F',
                    '[*]CCC(F)(F)C([*])(F)F'])[0].apply(Chem.MolFromSmiles)
MOL

0    <rdkit.Chem.rdchem.Mol object at 0x7f873772e430>
1    <rdkit.Chem.rdchem.Mol object at 0x7f873772e3c0>
2    <rdkit.Chem.rdchem.Mol object at 0x7f873772e580>
3    <rdkit.Chem.rdchem.Mol object at 0x7f873772e510>
4    <rdkit.Chem.rdchem.Mol object at 0x7f873772e660>
5    <rdkit.Chem.rdchem.Mol object at 0x7f873772e6d0>
Name: 0, dtype: object

In [13]:
#all_smile = pd.DataFrame(smiles)
#Mol = all_smile.squeeze()
MOL = smiles.apply(Chem.MolFromSmiles)

#print(MOL)
# MOL = Mol.apply(Chem.MolFromSmiles)
# type(Mol)
#df['smiles_mol'] = df['smiles'].apply(Chem.MolFromSmiles)
#df = df.dropna()

In [15]:
#fp_1 = df['smiles_mol'].apply(lambda m: AllChem.GetMorganFingerprint(m, radius=3))
fp_1 = MOL.apply(lambda m: AllChem.GetMorganFingerprint(m, radius=3))

fp_1_n = fp_1.apply(lambda m: m.GetNonzeroElements())
print(fp_1_n)


0       {755337623: 1, 1302073591: 1, 1307459754: 1, 1...
1       {188605169: 1, 485458970: 1, 882399112: 1, 130...
2       {71879828: 1, 485463469: 2, 882399112: 2, 1302...
3       {485458970: 1, 485463469: 2, 882399112: 3, 130...
4       {485463469: 4, 882399112: 4, 1302073591: 1, 13...
                              ...                        
6260    {412198813: 1, 485463469: 4, 666309351: 2, 712...
6261    {117706561: 1, 485463469: 4, 533204632: 1, 805...
6262    {485463469: 4, 595323049: 1, 712993388: 1, 864...
6263    {352073271: 1, 485463469: 4, 853890546: 1, 854...
6264    {18104817: 1, 280572389: 1, 485463469: 4, 5332...
Name: smiles, Length: 6265, dtype: object


In [16]:
# using substructures in dataset-1 to construct a dictionary
HashCode = []
for i in fp_1_n:
    for j in i.keys():
        HashCode.append(j)

unique_set = set(HashCode)
unique_list = list(unique_set)


Corr_df = pd.DataFrame(unique_list).reset_index()
# print(Corr_df)
# print(type(Corr_df))
print(Corr_df['index'].values)
#unique_list
print(len(unique_list))

[    0     1     2 ... 20691 20692 20693]
20694


In [17]:
#construct dataset-1 input
MY_finger = []
for polymer in fp_1_n:
    my_finger = [0] *len(unique_list)
    for key in polymer.keys():
        index = Corr_df[Corr_df[0] == key]['index'].values[0]
        my_finger[index] = polymer[key]
    MY_finger.append(my_finger)

MY_finger_dataset = pd.DataFrame(MY_finger)  
print(type(MY_finger_dataset))

<class 'pandas.core.frame.DataFrame'>


In [20]:
MY_finger_dataset
#print(MY_finger_dataset.to_string())


,0,1,2,3,4,5,6,7,8,9,...,20684,20685,20686,20687,20688,20689,20690,20691,20692,20693
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6260,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6261,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6262,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6263,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
print(type(MY_finger_dataset))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
MY_finger_dataset.to_csv('MY_finger_dataset')

In [22]:
# filter input into the most popular 124 substructures
Zero_Sum = (MY_finger_dataset == 0).astype(int).sum()
NumberOfZero = 6
print(len(Zero_Sum[Zero_Sum < NumberOfZero]))
X = MY_finger_dataset[Zero_Sum[Zero_Sum < NumberOfZero].index]

1


In [23]:
# Then, import the necessary modules
from sklearn import svm
from sklearn.model_selection import train_test_split
import random

# Generate random task labels 
labels = np.random.randint(0, 2, size=MY_finger_dataset.shape[0])

# # Load the task labels from the CSV file
# y = np.loadtxt("task_labels.csv", delimiter=",")

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(MY_finger_dataset, labels, test_size=0.2, random_state=42)




In [24]:
print(labels)
# Save the task labels to a CSV file
np.savetxt("task_labels.csv", labels, delimiter=",")

[1 0 0 ... 1 0 0]


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
# param_svm = {'kernel': ('linear', 'rbf','poly'),
#               'C':[1.5, 10],
#               'gamma': [1e-7, 1e-4],
#               'epsilon':[0.1,0.2,0.5,0.3]}
param_svm = {
              'C':[1, 10],
              'gamma': [0.1,1,10],
              }
svr = SVR()
random_svm = RandomizedSearchCV(svr,param_svm, cv= 5,scoring='accuracy')
random_svm.fit(X_train,y_train)
best_regressor_svm = random_svm.best_estimator_


/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:292: UserWarning: The total space of parameters 6 is smaller than n_iter=10. Running 6 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users

/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/Users/yuyang/miniconda3/lib/python3.9/site-packages/sklearn/metric

In [ ]:
y_pred_svm = best_regressor_svm.predict(X_test)
y_pred_svm_mean = np.mean(y_pred_svm)
print(y_pred_svm_mean)

In [ ]:
from sklearn.metrics import mean_squared_error
# Compute Root Mean Square Error(RMSE)

Rmse_pred_svm = np.sqrt(mean_squared_error(y_test,y_pred_svm))

print(Rmse_pred_svm)

In [ ]:
# Import necessary libraries
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error

# Define the kernel for the Gaussian process
kernel = C(1.0, (1e-3, 1e3)) * RBF(10, (1e-2, 1e2))

# Create the Gaussian process regressor object
gp = GaussianProcessRegressor(kernel=kernel)

# Define the parameter grid for the search
param_grid = {
    "kernel__k1__constant_value": [1e-1, 1, 1e1, 1e2],
    "kernel__k2__length_scale": [1e-2, 1e-1, 1, 1e1, 1e2],
}

# Create the RandomizedSearchCV object
Random_gpr = RandomizedSearchCV(
    gp, param_grid, cv=5, n_iter=10, scoring=mean_squared_error, random_state=42
)

# Fit the model to the data using the fingerprint vectors as input
Random_gpr.fit(X_train,y_train)

best_regressor_gpr = random_gpr.best_estimator_


/usr/local/lib/python3.8/dist-packages/sklearn/gaussian_process/kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__length_scale is close to the specified lower bound 0.01. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
TypeError: mean_squared_error() takes 2 positional arguments but 3 were given

  warnings.warn(


In [ ]:
# Use the model to make predictions
y_pred_gpr = best_regressor_gpr.predict(X_test)
y_pred_gpr_mean = np.mean(y_pred_gpr)
print(y_pred_gpr_mean)

In [ ]:
# Compute Root Mean Square Error(RMSE)

Rmse_pred_gpr = np.sqrt(mean_squared_error(y_test,y_pred_gpr))

print(Rmse_pred_gpr)

In [ ]:

from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV


# Create the multi-task learning model
model = MultiOutputRegressor()

# Use RandomizedSearchCV to tune the hyperparameters
param_distributions = {'n_estimators': [10, 50, 100, 200],
                       'max_depth': [None, 2, 4, 6, 8]}
Random_multi = RandomizedSearchCV(model, param_distributions, n_iter=10, cv=5)

# Fit the model to the training data
Random_multi.fit(X_train, y_train)

best_regressor_multi = random_multi.best_estimator_



In [ ]:
# Use the model to make predictions
y_pred_multi = best_regressor_multi.predict(fingerprint_vectors)
y_pred_multi_mean = np.mean(y_pred_gpr)
print(y_pred_multi_mean)

In [ ]:
# Compute Root Mean Square Error(RMSE)

Rmse_pred_multi = np.sqrt(mean_squared_error(y_test,y_pred_multi))

print(Rmse_pred_multi)

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Load the numerical vector representation of the Morgan Fingerprint and the labels for each task
X = np.load("fingerprint_data.npy")
y = np.load("task_labels.npy")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the neural network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(64, activation='relu', input_shape=(X.shape[1],)),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(y.shape[1], activation='linear')
])
model.compile(optimizer='adam', loss='mean_squared_error')

# Define the parameter distribution for the RandomizedSearchCV
param_distributions = {'batch_size': [16, 32, 64, 128],
                       'epochs': [10, 20, 50, 100]}

# Use RandomizedSearchCV to tune the hyperparameters of the neural network model
random_search = RandomizedSearchCV(model, param_distributions, cv=5, n_iter=10)

# Fit the model to the training data
random_search.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = random_search.predict(X_test)


In [ ]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Load the numerical vector representation of the Morgan Fingerprint and the labels for each task
X = np.load("fingerprint_data.npy")
y = np.load("task_labels.npy")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Define the Random Forest regressor
rf = RandomForestRegressor()

# Define the parameter distribution for the RandomizedSearchCV
param_distributions = {'n_estimators': [10, 50, 100, 200, 500],
                       'max_depth': [2, 5, 10, 20, None]}

# Use RandomizedSearchCV to tune the hyperparameters of the Random Forest regressor
random_search = RandomizedSearchCV(rf, param_distributions, cv=5, n_iter=10)

# Fit the Random Forest regressor to the training data
random_search.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = random_search.predict(X_test)


In [ ]:
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# Load the numerical vector representation of the Morgan Fingerprint and the labels for each task
X = np.load("fingerprint_data.npy")
y = np.load("task_labels.npy")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Convert the data to the format required by XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# Define the XGBoost model
xgboost = xgb.XGBRegressor()

# Define the parameter distribution for the RandomizedSearchCV
param_distributions = {'n_estimators': [10, 50, 100, 200, 500],
                       'max_depth': [2, 5, 10, 20]}

# Use RandomizedSearchCV to tune the hyperparameters of the XGBoost model
random_search = RandomizedSearchCV(xgboost, param_distributions, cv=5, n_iter=10)

# Fit the XGBoost model to the training data
random_search.fit(X_train, y_train)

# Make predictions on the testing data
y_pred = random_search.predict(X_test)
